In [8]:
import helperfuncs as hp
import numpy as np
from scipy.optimize import minimize
from scipy.special import comb
import scipy.stats

# Defining the PDF and the Log Likelihoods

Consider a random variable $z_i$ of length $d$ given by:

$$
z_i \sim \mathcal{N}(0, I + r_i S_i^{-1/2} V S_i^{-1/2})
$$

The log likelihood $l$ for a SNP $i$ is given by:

$$
l_i = -\frac{d}{2} log(2 \pi) - \frac{1}{2} log |I + r_i S_i^{-1/2} V S_i^{-1/2} | - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^ {-1}z_i
$$

Its derivative with respect to V is given by:

$$
\frac{dl_i}{V} =-\frac{1}{2}\left[(I + r_i S_i^{-1/2} V _i^{-1/2} S_i^{-1/2}) r_i S_i^{-1/2}S_i^{-1/2} + z_i z_i^T [-(I + r_i S_i^{-1/2} V S_i^{-1/2} ) r_i S_i^{-1/2} S_i^{-1/2} (I + r_i S_i^{-1/2} V S_i^{-1/2})]\right]
$$

# Still to develop

In [52]:
class sibreg():
    
    def __init__(self, S, z = None, u = None, r = None, f = None):
        
        if S.ndim > 1:
            for s in S:
                n, m = s.shape
                assert n == m

        if z is None:
            print("Warning there is no value for z. Maybe consider simulating it")
        if u is None:
            print("No value for U given. Generating a vector of ones (all SNPs weighted equally)")
            u = np.ones(S.shape[0])
        if r is None:
            print("No value for r given. Generating a vector of ones for r")
            r = np.ones(S.shape[0])
        if f is None:
            print("Warning: No value given for allele frequencies. Some parameters won't be noramlized.")
        
        self.z = None if z is None else z[~np.any(np.isnan(z), axis = 1)]
        self.S = S[~np.any(np.isnan(S), axis = (1, 2))]
        self.u = u[~np.isnan(u)]
        self.r = r[~np.isnan(r)]
        self.f = None if f is None else f[~np.isnan(f)]
    

    def simdata(self, V,  N, simr = False):
        
        # Simulated data (theta hats) as per section 7.1
        # V = varcov matrix of true effects
        # N = Number of obs/SNPs to generate
        
        S = self.S
        
        if simr:
            self.r = np.random.uniform(low=1, high=5, size=N)
            print("Simulated LD scores!")
        
        r = self.r

        zhat_vec = np.empty((N, V.shape[1]))
        for i in range(N):
            
            Si = S[i]
            ri = r[i]
            
            V = np.array(V)/N
            Si = np.array(Si)
            S_inv = np.linalg.inv(np.sqrt(Si))
  
            # get shape of V
            d = V.shape[0]
            zeromat = np.zeros(d)

            # generate true effect vector
            if d > 1:
                sim = np.random.multivariate_normal(zeromat, np.eye(d) + ri * S_inv @ V @ S_inv)
            else:
                sim = np.random.normal(zeromat, np.eye(d) + ri * S_inv @ V @ S_inv)
            
            # Append to vector of effects
            zhat_vec[i, :] = sim
        

        print("Effect Vectors Simulated!")
        
        self.snp = np.arange(1, N+1, 1)
        self.pos = np.arange(1, N+1, 1)
        self.z = zhat_vec

    def neg_logll_grad(self, V, z = None, S = None, u = None, r = None, f = None):
        
        # ============================================ #
        # returns negative log likelihood and negative
        # of the gradient
        # ============================================ #
        
        z = self.z if z is None else z
        S = self.S if S is None else S
        u = self.u if u is None else u
        r = self.r if r is None else r
        f = self.f if f is None else f

        # Unflatten V into a matrix
        d = S[0].shape[0]
        V = hp.return_to_symmetric(V, d)
        Gvec = np.zeros((d, d))
        
        N = len(S)
        log_ll = 0
        
        # Normalizing variables
        # V = V * N
        V_norm = V/N
        for i in range(N):
            
            Si = S[i]
            zi = z[i, :].reshape((d, 1))
            ui = u[i]
            ri = r[i]
            
            
            fi = f[i]  if f is not None else None

            d, ddash = Si.shape
            assert d == ddash # Each S has to be a square matrix
            
            # normalizing variables using allele frequency
#             normalizer = 2 * fi  * (1 - fi) if fi is not None else 1.0
#             z = np.sqrt(normalizer) * thetai
#             Si = normalizer * Si
            
            Si_inv = np.linalg.inv(np.sqrt(Si))
            sigma_exp = np.eye(d) + ri * Si_inv @ V_norm @ Si_inv
      
            # calculate log likelihood
            log_ll_add = -(d/2) * np.log(2 * np.pi)
            dit_sv = np.linalg.det(sigma_exp)
            dit_sv = 1e-6 if dit_sv < 0 else dit_sv
            log_ll_add += -(1/2) * np.log(dit_sv)
            log_ll_add += -(1/2) * np.trace(zi @ zi.T @ np.linalg.inv(sigma_exp))
            log_ll_add *= 1/ui
            
            if np.isnan(log_ll_add) == False:
                log_ll += log_ll_add
            
            # calculate gradient
            G = (-1/2) * (sigma_exp)
            G = G @ (ri * Si_inv @ Si_inv)
            G += zi @ zi.T @ (-(sigma_exp) \
                                 @ (ri * Si_inv @ Si_inv) \
                                 @ (sigma_exp))
            if np.any(np.isnan(G)) == False:
                Gvec += G

        Gvec = hp.extract_upper_triangle(Gvec)
        return -log_ll , -Gvec


    def solve(self,
              z = None, 
              S = None,
              u = None,
              r = None,
              f = None,
              neg_logll_grad = None,
              est_init = None,
              printout = True):
        
        # inherit parameters from the class if they aren't defined
        z = self.z if (z is None) else z
        S = self.S if S is None else S
        u = self.u if u is None else u
        r = self.r if r is None else r
        f = self.f if f is None else f
        neg_logll_grad = self.neg_logll_grad if neg_logll_grad is None else neg_logll_grad

        # == Solves our MLE problem == #
        n, m = z.shape
        
        if est_init is not None:
            # Shape of initial varcov guess
            rowstrue = est_init.shape[0] == m
            colstrue = est_init.shape[1] == m

            if rowstrue & colstrue:
                pass
            else:
                if printout == True:
                    print("Warning: Initial Estimate given is not of the proper dimension")
                    print("Making 'optimal' matrix")
                    print("=================================================")
                
                est_init = np.zeros((m, m))
        else:
            if printout == True:
                print("No initial guess provided.")
                print("Making 'optimal' matrix")
                print("=================================================")
            
            est_init = np.zeros((m, m))
            
        
        # exporting for potential later reference
        self.est_init = est_init

        # extract array from est init
        est_init_array = hp.extract_upper_triangle(est_init) 
        
        bounds = hp.extract_bounds(m)

        result = minimize(
            neg_logll_grad, 
            est_init_array,
            jac = True,
            args = (z, S, u, r, f),
            bounds = bounds,
            method = 'L-BFGS-B'
        )
        
        output_matrix = hp.return_to_symmetric(result.x, m)
        
        # re-normnalizing output matrix
        output_matrix = output_matrix / n
        
        self.output_matrix = output_matrix
        
        return output_matrix, result 

    def jackknife_se(self,
                  theta  = None, S = None,
                  r = None, u = None,
                  blocksize = 1):

        # Simple jackknife estimator for SE
        # Ref: https://github.com/bulik/ldsc/blob/aa33296abac9569a6422ee6ba7eb4b902422cc74/ldscore/jackknife.py#L231
        # Default value of blocksize = 1 is the normal jackknife

        theta = self.theta if (theta is None) else theta
        S = self.S if (S is None) else S
        r = self.r if (r is None) else r
        u = self.u if (u is None) else u

        
        assert theta.shape[0] == S.shape[0]

        nobs = theta.shape[0]
        
        estimates_jk = []
        
        start_idx = 0
        while True:
            
            end_idx = start_idx + blocksize
            end_idx_cond = end_idx <= theta.shape[0]
            
            # remove blocks of observations

            vars_jk = []

            for var in [theta, S, r, u]:

                var_jk = delete_obs_jk(var, start_idx, end_idx,
                                       end_idx_cond)
                vars_jk.append(var_jk)
            
            if start_idx < theta.shape[0]:
                # Get our estimate
                output_matrix, _ = self.solve(theta = vars_jk[0],
                                              S = vars_jk[1],
                                              r = vars_jk[2],
                                              u = vars_jk[3],
                                              printout = False,
                                              est_init = self.est_init)

                estimates_jk.append(output_matrix)

                start_idx += blocksize
            else:
                break
            
        estimates_jk = np.array(estimates_jk)
        full_est = self.output_matrix
        
        # calculate pseudo-values
        n_blocks = int(nobs/blocksize)
        pseudovalues = n_blocks * full_est - (n_blocks - 1) * estimates_jk
        
        # calculate jackknife se
        pseudovalues = pseudovalues.reshape((n_blocks, theta.shape[1] * theta.shape[1]))
        jknife_cov = np.cov(pseudovalues.T, ddof=1) / n_blocks
        jknife_var = np.diag(jknife_cov)
        jknife_se = np.sqrt(jknife_var)
    
        jknife_se  = jknife_se.reshape((theta.shape[1], theta.shape[1]))
        
        return jknife_se  


In [53]:
N = 100
S_size=  int(N/2)
S = np.array([np.array([[.5, 0], [0, .8]]),
    np.array([[0.5, 0], [0, 0.8]])] * S_size )/N
V = np.identity(2) * 0.5

In [54]:
model = sibreg(S = S)
model.simdata(V, N, simr = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [55]:
Vin = hp.extract_upper_triangle(V)
model.neg_logll_grad(Vin * N)

(728.9845921999076, array([9.33808771e+09, 7.38525333e+07, 2.58037617e+09]))

In [56]:
model.solve()

No initial guess provided.
Making 'optimal' matrix


(array([[ 1.00000000e-08, -6.01886222e-03],
        [-6.01886222e-03,  1.00000000e-08]]),       fun: -507.18498187357187
  hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 614699.10539998, -412926.16648053, 1002942.66333233])
   message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
      nfev: 63
       nit: 2
    status: 2
   success: False
         x: array([ 1.00000000e-06, -6.01886222e-01,  1.00000000e-06]))

In [6]:
def logll(V):

    Vin = hp.extract_upper_triangle(V)
    logll = -model.neg_logll_grad(Vin)[0]
    
    return logll

def gradlogll(V):

    Vin = hp.extract_upper_triangle(V)
    grad = -model.neg_logll_grad(Vin)[1]
    
    return grad

def num_grad(V, row = 0, col = 0, dx = 1e-6):
    
    V1 = np.copy(V)
    V1[row, col] += dx
    
    derivative = (logll(V1) - logll(V))/dx
    
    return derivative

num_grad(V, 0, 0)

array([[-51.6393107]])

In [7]:
gradlogll(V)

array([-1548235.57487135,    11072.70223056,  -556545.8429542 ])

The gradients dont align!

Let's see if the likelihood is correctly specified.

In [27]:
Si = S[0]
ri = model.r[0]
S_inv = np.linalg.inv(np.sqrt(Si))
logll = scipy.stats.multivariate_normal.logpdf(model.z, 
                                       mean = np.zeros(2),
                                      cov = np.eye(2) + ri * S_inv @ V/N @ S_inv)
logll.sum()

-359.25346977141857

In [32]:
Vin = hp.extract_upper_triangle(V)
-model.neg_logll_grad(Vin)[0][0, 0]

-341.5283683896606